In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tqdm
import selfies as sf
from rdkit import Chem

SCORING_LIB_PATH = os.path.join('/Data1', 'sungmin', 'mo_vq', 'props')
if SCORING_LIB_PATH not in sys.path:
    sys.path = [SCORING_LIB_PATH] + sys.path
    
from properties import similarity, drd2, qed

In [2]:
class CONFIGS:
    HOME_PATH = os.path.join('/Data1', 'sungmin', 'fairseq')
    DATA_TYPE = "qed"
    TOKEN_TYPE = "frag"
    NUM_IDX = 5
    
    FILE_NAME = 'drd2_{NUM}.csv' if DATA_TYPE == 'drd2' else 'qed_v{NUM}.csv'
    
    OUTPUT_DIR = 'outputs_1_evaluate_property_similarity_scores'
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    OUTPUT_DIR = os.path.join(OUTPUT_DIR, DATA_TYPE)
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)
    
configs = CONFIGS()

In [3]:
calc_prop = drd2 if configs.DATA_TYPE == 'drd2' else qed

## Generated molecules

In [4]:
def get_canonical(sel):
    return Chem.CanonSmiles(sf.decoder(sel))

In [5]:
frames_generated = []
for i in tqdm.trange(configs.NUM_IDX):
    filepath_generated = os.path.join(configs.HOME_PATH, 'fairseq_cli', 'result', configs.FILE_NAME)
    df_generated = pd.read_csv(filepath_generated.format(NUM=i+1), index_col=0)
    df_generated.loc[:,'SELFIES_ori'] = df_generated.loc[:,'original'].map(lambda x:x.replace(' ',''))
    df_generated.loc[:,'SELFIES_opt'] = df_generated.loc[:,'optimized'].map(lambda x:x.replace(' ',''))
    df_generated.loc[:,'SELFIES_tar'] = df_generated.loc[:,'target'].map(lambda x:x.replace(' ',''))
    df_generated.loc[:,'SMILES_ori'] = df_generated.loc[:,'SELFIES_ori'].map(get_canonical)
    df_generated.loc[:,'SMILES_opt'] = df_generated.loc[:,'SELFIES_opt'].map(get_canonical)
    df_generated.loc[:,'SMILES_tar'] = df_generated.loc[:,'SELFIES_tar'].map(get_canonical)
    frames_generated.append(df_generated)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:26<00:00, 17.34s/it]


In [6]:
frames_generated[0]

,original,optimized,target,SELFIES_ori,SELFIES_opt,SELFIES_tar,SMILES_ori,SMILES_opt,SMILES_tar
0,[N][C] [C][C][N][Branch1][#Branch2][C][C][N][R...,[N][C] [C][C][N][Branch1][#Branch2][C][C][N][R...,[N][C] [C][C][N][Branch1][#Branch2][C][C][N][R...,[N][C][C][C][N][Branch1][#Branch2][C][C][N][Ri...,[N][C][C][C][N][Branch1][#Branch2][C][C][N][Ri...,[N][C][C][C][N][Branch1][#Branch2][C][C][N][Ri...,CC[NH2+]Cc1c(C)nn(C)c1N1CCN2C(=O)NCC2C1,COc1cc(C)c(CN2CCN3C(=O)NCC3C2)cc1OC,COc1cc(C)c(CN2CCN3C(=O)NCC3C2)cc1OC
1,[C] [Branch1][=Branch2][NH2+1][C][C][S][Branch...,[C] [Branch1][P][C][C][O][C][Branch1][Ring2][C...,[C] [Branch1][P][C][C][O][C][Branch1][Ring2][C...,[C][Branch1][=Branch2][NH2+1][C][C][S][Branch1...,[C][Branch1][P][C][C][O][C][Branch1][Ring2][C]...,[C][Branch1][P][C][C][O][C][Branch1][Ring2][C]...,CS(=O)CC[NH2+]CC=Cc1ccc(C#N)cc1,N#Cc1ccc(C=CC[NH2+]C2CCOC3(CCCCC3)C2)cc1,N#Cc1ccc(C=CC[NH2+]C2CCOC3(CCCCC3)C2)cc1
2,[C][Branch1][=Branch2][O][C][C][NH1+1][Branch1...,[C][=C][C][=Branch1][#C][=C][C][=C][Ring1][=Br...,[C][=C][C][=Branch1][#C][=C][C][=C][Ring1][=Br...,[C][Branch1][=Branch2][O][C][C][NH1+1][Branch1...,[C][=C][C][=Branch1][#C][=C][C][=C][Ring1][=Br...,[C][=C][C][=Branch1][#C][=C][C][=C][Ring1][=Br...,C[NH+](C)CCOc1ccc(CNC(=O)C2COc3ccc(F)cc3C2)cc1,O=C(NCc1ccc(OC(F)(F)F)cc1)C1COc2ccccc2C1,O=C(NCc1ccc(OC(F)(F)F)cc1)C1COc2ccccc2C1
3,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,[C][Branch1][Ring1][C][#N][Branch1][#Branch1][...,CC1CCC(O)(C2(C#N)CCOC2)CC1C,N#CC1(C(O)c2cc(Cl)cc(Cl)c2)CCOC1,N#CC1(C(O)c2cc(Cl)cc(Cl)c2)CCOC1
4,[C][C][=Branch1][C][=O][N][Branch1][C][C][C][C...,[C][C][Branch1][#Branch2][C][C][=Branch1][C][=...,[C][C][Branch1][#Branch2][C][C][=Branch1][C][=...,[C][C][=Branch1][C][=O][N][Branch1][C][C][C][C...,[C][C][Branch1][#Branch2][C][C][=Branch1][C][=...,[C][C][Branch1][#Branch2][C][C][=Branch1][C][=...,COc1cc(C(=O)NC2CC(=O)N(C)C2)nc(OC)n1,CN1CC(NC(=O)c2ccc(F)c(Br)c2)CC1=O,CN1CC(NC(=O)c2ccc(F)c(Br)c2)CC1=O
...,...,...,...,...,...,...,...,...,...
15995,[C][=C] [Branch1][C][Cl] [C][=C] [Branch1][C][...,[C][=C] [Branch1][C][Cl] [C][=C] [Branch1][C][...,[C][=C] [Branch1][C][Cl] [C][=C] [Branch1][C][...,[C][=C][Branch1][C][Cl][C][=C][Branch1][C][F][...,[C][=C][Branch1][C][Cl][C][=C][Branch1][C][F][...,[C][=C][Branch1][C][Cl][C][=C][Branch1][C][F][...,O=C(CCCn1c(=O)oc2ccccc21)Nc1ccc(Cl)cc1F,CC1Cc2ccccc2N1CC(=O)Nc1ccc(Cl)cc1F,CC1Cc2ccccc2N1CC(=O)Nc1ccc(Cl)cc1F
15996,[C][C][C] [Branch1][N] [C][=C][C][=C][Branch1]...,[C][=C][C][Branch1][C][C][=C][C][=C][Ring1][#B...,[C][=C][C][Branch1][C][C][=C][C][=C][Ring1][#B...,[C][C][C][Branch1][N][C][=C][C][=C][Branch1][C...,[C][=C][C][Branch1][C][C][=C][C][=C][Ring1][#B...,[C][=C][C][Branch1][C][C][=C][C][=C][Ring1][#B...,CCC(NC(=O)c1cccc(S(=O)(=O)NC(C)C)c1)c1ccc(C)cc1,Cc1ccc(-c2ccccc2NC(=O)Cn2cnnn2)cc1,CCC(NC(=O)c1cccc(S(C)(=O)=O)c1)c1ccc(C)cc1
15997,[C] [Branch1] [C][F][=C][C][Branch1][C][Br][=C...,[C] [=Branch1] [N][=C][C][Branch1][C][C][=C][C...,[C] [=Branch1] [N][=C][C][Branch1][C][C][=C][C...,[C][Branch1][C][F][=C][C][Branch1][C][Br][=C][...,[C][=Branch1][N][=C][C][Branch1][C][C][=C][C][...,[C][=Branch1][N][=C][C][Branch1][C][C][=C][C][...,Cc1ccccc1CS(=O)(=O)N1CCCC(C(=O)Nc2ccc(Br)cc2F)C1,Cc1ccc(F)c(NC(=O)C2CCCN(S(C)(=O)=O)C2)c1,Cc1ccc(F)c(NC(=O)C2CCCN(S(C)(=O)=O)C2)c1
15998,[S] [Branch1][C][C] [=Branch1][C][=O] [=Branch...,[C] [Branch1][C][C] [S] [=Branch1][C][=O] [=Br...,[C] [Branch1][C][C] [S] [=Branch1][C][=O] [=Br...,[S][Branch1][C][C][=Branch1][C][=O][=Branch1][...,[C][Branch1][C][C][S][=Branch1][C][=O][=Branch...,[C][Branch1][C][C][S][=Branch1][C][=O][=Branch...,C[NH+](C)C1CCC(NC(=O)c2ccc3c(c2)CCCN3S(C)(=O)=...,CCS(=O)(=O)N1CCCc2cc(C(=O)C3CC(c4nc(C)n[nH]4)C...,CCS(=O)(=O)N1CCCc2cc(C(=O)NC3CC3)ccc21


## Evaluate property and simiarity scores

In [7]:
frames_scoring = []
for i, df_ in enumerate(frames_generated):
    print(f'Now: {i}')
    df = df_.copy()
    
    scores_prop = []
    for smi_opt in tqdm.tqdm(df.loc[:,'SMILES_opt'].values, total=len(df)):
        scores_prop.append(calc_prop(smi_opt))
    
    scores_sim = []
    for smi_ori, smi_opt in tqdm.tqdm(df.loc[:,('SMILES_ori', 'SMILES_opt')].values, total=len(df)):
        scores_sim.append(similarity(smi_ori, smi_opt))
        
    df.loc[:,'PROPERTY_opt'] = scores_prop
    df.loc[:,'SIMILARITY_ori_opt'] = scores_sim
    frames_scoring.append(df)

Now: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3585.61it/s]


Now: 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3616.74it/s]


Now: 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3621.06it/s]


Now: 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3556.74it/s]


Now: 4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [00:04<00:00, 3651.42it/s]


In [8]:
for i, df in enumerate(frames_scoring, start=1):
    filepath = os.path.join(configs.OUTPUT_DIR, f'generated_edit_iter{i}.csv')
    df.to_csv(filepath, index=False)